In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from copy import copy

from sklearn import svm
from sklearn.metrics import f1_score
from sklearn.model_selection import KFold

In [2]:
from collections import defaultdict
from scipy.spatial import distance
from sklearn.metrics import pairwise_distances
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
from bs4 import BeautifulSoup as bs

In [3]:
import nltk
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation
from nltk.stem.snowball import SnowballStemmer
import pandas as pd
import numpy as np
import re

In [34]:
# nltk.download()

In [15]:
lem_titles = pd.read_csv("lemm_titles.csv")
train_groups = pd.read_csv("train_groups.csv")
test_groups = pd.read_csv("test_groups.csv")
lemm_titles_test = pd.read_csv("lemm_titles_test.csv")
titles = pd.read_csv("new_titles.csv")

In [4]:
import multiprocessing

In [8]:
res = []
pool = multiprocessing.Pool(4)
def func(i):
    with open(f"content/{i}.dat", "r") as f:
        file = f.read()
        soup = bs(file)
        description = soup.find("meta", attrs={"name":"description"})
        title = soup.title.text
        print(i, end="\r")
        return[i, title, None if description is None else description.get("content")]

res = [*pool.map(func, range(1, 28027))]

Process ForkPoolWorker-20:
Process ForkPoolWorker-18:
Process ForkPoolWorker-19:
Process ForkPoolWorker-17:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/pool.py", line 110, in worker
    task = get()
  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/home/maksim/minico

KeyboardInterrupt


  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/queues.py", line 352, in get
    res = self._reader.recv_bytes()
  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/connection.py", line 216, in recv_bytes
    buf = self._recv_bytes(maxlength)


  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/connection.py", line 407, in _recv_bytes
    buf = self._recv(4)
  File "/home/maksim/miniconda3/lib/python3.7/multiprocessing/connection.py", line 379, in _recv
    chunk = read(handle, remaining)
KeyboardInterrupt


In [36]:
def transform_title(title, prep=None):
    title = re.sub(r"[^\w\d' ']", " ", str(title).lower())
    mystem = Mystem() 
    russian_stopwords = stopwords.words("russian")
    english_stopwords = stopwords.words("english")
    if prep is None:
        tokens = title.split()
    elif prep == 'stem':
        stemmerR = SnowballStemmer("russian")
        stemmerE = SnowballStemmer("english")
        tokens = map(lambda x: stemmerR.stem(stemmerE.stem(x)), title.split())
    elif prep == 'lem':
        tokens = mystem.lemmatize(title.lower())
    tokens = [token for token in tokens if token not in russian_stopwords
              and token != " " and token not in english_stopwords
              and token.strip() not in punctuation]
    
    title = " ".join(tokens)
    
    return title

In [53]:
df = pd.DataFrame(res, columns=["id", "title", "description"])

In [12]:
df.to_csv("new_titles.csv", index=False)

In [5]:
words_train = lem_titles["title"]
words_test = lemm_titles_test["title"]

data_train = ["" if i is np.nan else i for i in words_train]
data_test = ["" if i is np.nan else i for i in words_test]
tfidf = TfidfVectorizer(ngram_range=(1, 1))
tf_data = tfidf.fit_transform(data_train + data_test)
tf_data

<28317x27276 sparse matrix of type '<class 'numpy.float64'>'
	with 182341 stored elements in Compressed Sparse Row format>

In [6]:
group_id_train = [i for i in train_groups["group_id"]]
y_train = np.array([i for i in train_groups["target"]])

group_id_test = [i for i in test_groups["group_id"]]

In [7]:
a = defaultdict(list)

In [8]:
for pair_id, group_id in zip(train_groups["pair_id"], train_groups["group_id"]):
    a[group_id].append(pair_id)
for pair_id, group_id in zip(test_groups["pair_id"], test_groups["group_id"]):
    a[group_id].append(pair_id)


In [66]:
db_scans = []
X_new = []
for group_id, values in a.items():
    d = pairwise_distances(tf_data[np.array(values) - 1], metric="cosine")
    dn = np.array(d)
    dn = np.sort(dn, axis=1)[:, 1:25]
    mean = np.mean(dn)
    percs = [np.percentile(dn, p) for p in range(25, 100, 25)]
    dbscan = DBSCAN(eps=0.001, metric="cosine")
    clusters = np.array(dbscan.fit_predict(dn))
    db_scans.append(clusters)
    
    # cluster means
    u_clusters = np.unique(clusters)
    means_clusters_vec = np.ones(shape=len(values), dtype=float)
    for cluster in u_clusters:
        indx = (clusters == cluster)
        means_clusters_vec[indx] *= np.mean(dn[indx])
    
    if np.max(np.abs(clusters)) != 0:
        clusters = clusters / np.max(np.abs(clusters))
    for v_id, i in enumerate(d):
        sorted_arr = sorted(i)[1:25]
        stats = np.array([mean] + percs + [clusters[v_id], means_clusters_vec[v_id]])
        sorted_arr = np.concatenate([sorted_arr, stats])
        X_new.append(sorted_arr)
        
X_new = np.array(X_new)

In [67]:
X_train = X_new[:len(data_train)]
X_test = X_new[len(data_train):]

In [68]:
X_test.shape

(16627, 30)

Попробуем добавить фичи

In [75]:
class combined_cls:
    def __init__(self, clses, result_cls):
        self.clses = clses
        self.result_cls = result_cls
        
    def fit(self, X, y):
        for model in self.clses:
            model.fit(X, y)
        new_X = np.concatenate([X] + [model.predict(X).reshape(-1, 1) for model in self.clses], axis=1)
#         new_X = np.concatenate([model.predict(X).reshape(-1, 1) for model in self.clses], axis=1)
        self.result_cls.fit(new_X, y)
        
    def predict(self, X):
        X = np.concatenate([X] + [model.predict(X).reshape(-1, 1) for model in self.clses], axis=1)
        return self.result_cls.predict(X)

In [76]:
tmp = []
for train_index, test_index in KFold(5).split(X_train, y_train):
    clf = combined_cls([RandomForestClassifier(max_depth=8, n_estimators=400),
                       svm.SVC(C=1, gamma=0.001, kernel="rbf", class_weight="balanced", probability=True)],
                      svm.SVC(C=1, gamma=0.001, kernel="rbf", class_weight="balanced"))
    clf.fit(X_train[train_index], y_train[train_index])
    tmp.append(f1_score(y_train[test_index], clf.predict(X_train[test_index])))
tmp

[0.7566765578635014,
 0.7580645161290323,
 0.7559808612440192,
 0.6814814814814815,
 0.8181253879577902]

In [77]:
np.mean(tmp)

0.754065760935165

In [78]:
clf = combined_cls([RandomForestClassifier(max_depth=8, n_estimators=500),
                   svm.SVC(C=1, gamma=0.001, kernel="rbf", class_weight="balanced")],
                  svm.SVC(C=1, gamma=0.001, kernel="rbf", class_weight="balanced"))
clf.fit(X_train, y_train)
result = clf.predict(X_test)

In [79]:
df = pd.DataFrame({"pair_id": [i for i in test_groups["pair_id"]], "target": result})

In [80]:
df.to_csv("res.csv", index=False)